<p><img alt="UNQ logo" height="45px" src="http://www.unq.edu.ar/images/logo_unqui_footer.png" align="left" hspace="10px" vspace="0px"></p><h1>Redes Neuronales y Lógica Difusa</h1>

En esta clase vamos a hacer uso de un módulo muy potente y útil llamado [*Tensorflow*](https://www.tensorflow.org/learn). Si bien su uso puede ser un poco menos natural para quienes están llegando a **Python** viniendo de herramientas como **MATLAB**, vamos a intentar en esta clase mostrar algunos ejemplos básicos de uso para que se comprenda su funcionamiento.

Vamos a introducir también en la clase a un nuevo módulo para la materia llamado [*Pandas*](https://pandas.pydata.org/), el cual nos permite manejar datos de forma potente, fácil y sencilla.

## Back-propagation serie 2

Cargo [*Tensorboard*](https://www.tensorflow.org/tensorboard), la herramienta gráfica de visualización de modelos de *Tensorflow*.

In [ ]:
%load_ext tensorboard

Borro archivos de logs de sesiones previas

In [ ]:
!rm -rf ./logs/

Importo módulos a utilizar

In [ ]:
import numpy as np
import pandas as pd
import datetime

from tensorflow import keras
from tensorflow.keras import layers

Genero las series temporales

In [ ]:
# Ejemplo de Behera
sigmaRuido = 0.2

# Genero serie temporal
t = np.zeros((1, 1000))
u = np.zeros((1, 1000))
d = np.zeros((1, 1000))
d[0, 0] = 0.5
p = np.zeros((2, 1000))

for k in range(1000-1):
    u[0, k] = 6 * np.random.rand() - 3
    d[0, k + 1] = d[0, k]/(1 + d[0, k]**2) + u[0, k]**3
    p[0, k] = u[0, k]
    p[1, k] = d[0, k]
    t[0, k] = d[0, k + 1] + np.random.normal(0, sigmaRuido)

Creo el dataframe y miro los primeros datos para ver que tenga la forma esperada

In [ ]:
dataset = pd.DataFrame({'Entrada 1': p[0, :], 'Entrada 2': p[1,:], 'Salida': t[0, :]})

dataset.head()

Separo los datos de entrenamiento de los de pruebas

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print(train_dataset)

Veo las estadísticas de los datos

In [ ]:

train_stats = train_dataset.describe()
train_stats.pop("Salida")
train_stats = train_stats.transpose()
train_stats

Separo el valor objetivo, o la "etiqueta" de las características. Esta etiqueta es el valor que entrenará el modelo para predecir.

In [ ]:
train_labels = train_dataset.pop('Salida')
test_labels = test_dataset.pop('Salida')

Normalizo los datos. Es una buena práctica normalizar funciones que utilizan diferentes escalas y rangos. Aunque el modelo * podría * converger sin normalización de características, dificulta el entrenamiento y hace que el modelo resultante dependa de la elección de las unidades utilizadas en la entrada.

**Nota:** Aunque generamos intencionalmente estas estadísticas solo del conjunto de datos de entrenamiento, estas estadísticas también se utilizarán para normalizar el conjunto de datos de prueba. Necesitamos hacer eso para proyectar el conjunto de datos de prueba en la misma distribución en la que el modelo ha sido entrenado.

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

Listo las opciones de función de activación que tiene el módulo 

In [ ]:
dir(keras.activations)
# Puedo encontrar información de cada uno en:
# https://keras.io/api/layers/activations/

Listo las opciones de función de activación que tiene el módulo 

In [ ]:
dir(keras.optimizers)
# Puedo encontrar información de cada uno en:
# https://keras.io/api/optimizers/

Construyo el modelo

In [ ]:
model = keras.Sequential([
  layers.Dense(10, activation='sigmoid', input_shape=[len(train_dataset.keys())]),
  layers.Dense(10, activation='tanh'),
  layers.Dense(1)
])

optimizer = keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse'])

Inspecciono el modelo

In [ ]:
model.summary()

Me fijo si la salida tiene la forma esperada

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

Entreno el modelo durante 1000 épocas y registro la precisión de entrenamiento y validación en el objeto history. Defino un parámetro de paciencia, para dejar de entrenar luego de 10 iteraciones sin una mejora.


Separo los datos de validación de los de entrenamiento.

**NOTA:** Cada vez que lo desee, ejecuto el siguiente código para borrar los logs de sesiones previas.

In [ ]:
# !rm -rf ./logs/ 

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2,
  callbacks=[early_stop, tensorboard_callback])

Ejecuto la herramienta gráfica *Tensorboard* para evaluar el comportamiento del modelo.

In [ ]:
%tensorboard --logdir logs/fit

Más allá de la herramienta, puedo analizar la información del modelo evaluando el objeto history.

Miro los últimos elementos del entrenamiento

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Probamos ahora cómo predice el modelo para los datos de testeo que fueron separados antes del entrenamiento.

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))

También podemos verificar la red haciendo nuestros propios gráficos con el módulo *matplotlib*, como veníamos haciendo hasta ahora.

Grafico el error y el error cuadrático

In [ ]:
import matplotlib.pyplot as plt

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error')
plt.plot(hist['epoch'], hist['mae'],
          label='Train Error')
plt.plot(hist['epoch'], hist['val_mae'],
          label = 'Val Error')
plt.ylim([0,5])
plt.legend()

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error [$^2$]')
plt.plot(hist['epoch'], hist['mse'],
          label='Train Error')
plt.plot(hist['epoch'], hist['val_mse'],
          label = 'Val Error')
plt.ylim([0,20])
plt.legend()
plt.show()

Hago predicciones utilizando el conjunto de prueba.

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


Hago un histograma para graficar la distribución de errores.

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

## Ejemplo XOR

Cargo [*Tensorboard*](https://www.tensorflow.org/tensorboard), la herramienta gráfica de visualización de modelos de *Tensorflow*.

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Borro logs de sesiones previas

In [ ]:
!rm -rf ./logs/

Importo módulos a utilizar

In [ ]:
import numpy as np
import pandas as pd
import datetime

from tensorflow import keras
from tensorflow.keras import layers

Genero las series temporales

In [ ]:
p = np.zeros((2, 4))
t = np.zeros((1, 4))
p[0] = [0, 1, 0, 1]
p[1] = [0, 0, 1, 1]
t[0] = [0, 1, 1, 0]

Creo el dataframe y miro los primeros datos para ver que tenga la forma esperada

In [ ]:
dataset = pd.DataFrame({'Entrada 1': p[0, :], 'Entrada 2': p[1,:], 'Salida': t[0, :]})

dataset.head()

Separo los datos de entrenamiento de los de pruebas.

**NOTA:** En este caso usamos todos los datos para entrenar.

In [ ]:
train_dataset = dataset.sample(frac=1.0,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print(train_dataset)

Separo el valor objetivo, o la "etiqueta" de las características. Esta etiqueta es el valor que entrenará el modelo para predecir.

In [ ]:
train_labels = train_dataset.pop('Salida')
test_labels = test_dataset.pop('Salida')

Listo las opciones de función de activación que tiene el módulo 

In [ ]:
dir(keras.activations)
# Puedo encontrar información de cada uno en:
# https://keras.io/api/layers/activations/

Listo las opciones de función de activación que tiene el módulo 

In [ ]:
dir(keras.optimizers)
# Puedo encontrar información de cada uno en:
# https://keras.io/api/optimizers/

Construyo el modelo

In [ ]:
model = keras.Sequential([
    layers.Dense(2, activation='tanh', input_shape=[len(train_dataset.keys())]),
  layers.Dense(1)
])

optimizer = keras.optimizers.Adam(0.2)

model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse'])

Inspecciono el modelo

In [ ]:
model.summary()

Me fijo si la salida tiene la forma esperada

In [ ]:
example_batch = train_dataset[:10]
example_result = model.predict(example_batch)
example_result

Entreno el modelo durante 1000 épocas y registro la precisión de entrenamiento y validación en el objeto history. Defino un parámetro de paciencia, para dejar de entrenar luego de 10 iteraciones sin una mejora.

Separo los datos de validación de los de entrenamiento.


**Nota:** En este caso usamos todos los datos para entrenar.

Cada vez que lo desee, ejecuto el siguiente código para borrar los logs de sesiones previas.

In [ ]:
# !rm -rf ./logs/ 

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

EPOCHS = 150

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0,
  callbacks=[early_stop, tensorboard_callback])

Ejecuto una herramienta gráfica para evaluar el comportamiento del modelo.

In [ ]:
%tensorboard --logdir logs/fit

Más allá de la herramienta, puedo analizar la información del modelo evaluando el objeto history.

Miro los últimos elementos del entrenamiento

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Hago predicciones utilizando los datos de entrenamiento


In [ ]:
resultado = model.predict(train_dataset)
print(train_dataset)
print("\nResultado\n" + str(np.round_(resultado, 1)))
print("\nValor esperado")
print(train_labels)

## Bibliografía

La base la tomé de:

https://www.tensorflow.org/tensorboard/get_started

https://www.tensorflow.org/tutorials/keras/regression

https://towardsdatascience.com/building-a-deep-learning-model-using-keras-1548ca149d37